In [ ]:
# | default_exp _testing.in_memory_broker

In [ ]:
# | export


import asyncio
import hashlib
import random
import string
import uuid
from contextlib import contextmanager
from dataclasses import dataclass, field
from typing import *

from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from aiokafka.structs import ConsumerRecord, RecordMetadata, TopicPartition

import fastkafka._application.app
import fastkafka._components.aiokafka_consumer_loop
from fastkafka._components.logger import get_logger
from fastkafka._components.meta import (
    _get_default_kwargs_from_sig,
    classcontextmanager,
    delegates,
    patch,
)

In [ ]:
import random
import unittest
from contextlib import asynccontextmanager

import pytest

from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | export

logger = get_logger(__name__)

# Local Kafka broker
> In-memory mockup of Kafka broker protocol

## Kafka partition

In [ ]:
# | export


@dataclass
class KafkaRecord:
    topic: str = ""
    partition: int = 0
    key: Optional[bytes] = None
    value: bytes = b""
    offset: int = 0
    timestamp = 0
    timestamp_type = 0
    checksum = 0
    serialized_key_size = 0
    serialized_value_size = 0
    headers: Sequence[Tuple[str, bytes]] = field(default_factory=list)

In [ ]:
# | export


class KafkaPartition:
    def __init__(self, *, partition: int, topic: str):
        """
        Initialize a KafkaPartition object.

        Args:
            partition: The partition number.
            topic: The topic name.
        """
        self.partition = partition
        self.topic = topic
        self.messages: List[KafkaRecord] = list()

    def write(self, value: bytes, key: Optional[bytes] = None) -> RecordMetadata:  # type: ignore
        """
        Write a Kafka record to the partition.

        Args:
            value: The value of the record.
            key: The key of the record.

        Returns:
            The record metadata.
        """
        record = KafkaRecord(
            topic=self.topic,
            partition=self.partition,
            value=value,
            key=key,
            offset=len(self.messages),
        )
        record_meta = RecordMetadata(
            topic=self.topic,
            partition=self.partition,
            topic_partition=TopicPartition(topic=self.topic, partition=self.partition),
            offset=len(self.messages),
            timestamp=1680602752070,
            timestamp_type=0,
            log_start_offset=0,
        )
        self.messages.append(record)
        return record_meta

    def read(self, offset: int) -> Tuple[List[KafkaRecord], int]:
        """
        Read Kafka records from the partition starting from the given offset.

        Args:
            offset: The starting offset.

        Returns:
            A tuple containing the list of records and the current offset.
        """
        return self.messages[offset:], len(self.messages)

    def latest_offset(self) -> int:
        """
        Get the latest offset of the partition.

        Returns:
            The latest offset.
        """
        return len(self.messages)

In [ ]:
partition_index = 0
topic = "test"
partition = KafkaPartition(partition=partition_index, topic=topic)

msgs = [b"some_msg" for _ in range(25)]

expected = [
    KafkaRecord(topic=topic, partition=partition_index, value=msg, offset=offset)
    for offset, msg in enumerate(msgs)
]

for msg in msgs:
    partition.write(msg)

for offset in [0, 10, 20]:
    actual = partition.read(offset=offset)

    assert actual == (expected[offset:], len(msgs))

In [ ]:
partition_index = 0
topic = "test"
key = b"some_key"
partition = KafkaPartition(partition=partition_index, topic=topic)

msgs = [b"some_msg" for _ in range(25)]
expected = [
    KafkaRecord(
        topic=topic, partition=partition_index, value=msg, key=key, offset=offset
    )
    for offset, msg in enumerate(msgs)
]

for msg in msgs:
    partition.write(msg, key=key)

for offset in [0, 10, 20]:
    actual = partition.read(offset=offset)

    assert actual == (expected[offset:], len(msgs)), print(f"{actual} != {expected}")

## Kafka topic

In [ ]:
# | export


class KafkaTopic:
    def __init__(self, topic: str, num_partitions: int = 1):
        """
        Initialize a KafkaTopic object.

        Args:
            topic: The topic name.
            num_partitions: The number of partitions in the topic (default: 1).
        """
        self.topic = topic
        self.num_partitions = num_partitions
        self.partitions: List[KafkaPartition] = [
            KafkaPartition(topic=topic, partition=partition_index)
            for partition_index in range(num_partitions)
        ]

    def read(  # type: ignore
        self, partition: int, offset: int
    ) -> Tuple[TopicPartition, List[KafkaRecord], int]:
        """
        Read records from the specified partition and offset.

        Args:
            partition: The partition index.
            offset: The offset from which to start reading.

        Returns:
            A tuple containing the topic partition, list of Kafka records, and the new offset.
        """
        topic_partition = TopicPartition(topic=self.topic, partition=partition)
        records, offset = self.partitions[partition].read(offset)
        return topic_partition, records, offset

    def write_with_partition(  # type: ignore
        self,
        value: bytes,
        partition: int,
    ) -> RecordMetadata:
        """
        Write a record with a specified partition.

        Args:
            value: The value of the record.
            partition: The partition to write the record to.

        Returns:
            The metadata of the written record.
        """
        return self.partitions[partition].write(value)

    def write_with_key(self, value: bytes, key: bytes) -> RecordMetadata:  # type: ignore
        """
        Write a record with a specified key.

        Args:
            value: The value of the record.
            key: The key of the record.

        Returns:
            The metadata of the written record.
        """
        partition = int(hashlib.sha256(key).hexdigest(), 16) % self.num_partitions
        return self.partitions[partition].write(value, key=key)

    def write(  # type: ignore
        self,
        value: bytes,
        *,
        key: Optional[bytes] = None,
        partition: Optional[int] = None,
    ) -> RecordMetadata:
        """
        Write a record to the topic.

        Args:
            value: The value of the record.
            key: The key of the record (optional).
            partition: The partition to write the record to (optional).

        Returns:
            The metadata of the written record.
        """
        if partition is not None:
            return self.write_with_partition(value, partition)

        if key is not None:
            return self.write_with_key(value, key)

        partition = random.randint(0, self.num_partitions - 1)  # nosec
        return self.write_with_partition(value, partition)

    def latest_offset(self, partition: int) -> int:
        """
        Get the latest offset of a partition.

        Args:
            partition: The partition index.

        Returns:
            The latest offset of the partition.
        """
        return self.partitions[partition].latest_offset()

In [ ]:
msg = b"msg"

topic = KafkaTopic("test_topic", 1)

expected = RecordMetadata(
    topic="test_topic",
    partition=0,
    topic_partition=TopicPartition(topic="test_topic", partition=0),
    offset=0,
    timestamp=1680602752070,
    timestamp_type=0,
    log_start_offset=0,
)
actual = topic.write(msg)

assert expected == actual

expected = RecordMetadata(
    topic="test_topic",
    partition=0,
    topic_partition=TopicPartition(topic="test_topic", partition=0),
    offset=1,
    timestamp=1680602752070,
    timestamp_type=0,
    log_start_offset=0,
)
actual = topic.write(msg, key=b"123")

assert expected == actual, actual

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 10

topic = KafkaTopic(topic_name, partition_num)

# write to topic
for msg in msgs:
    topic.write(msg)

# For each partition in topic check:
for partition in range(partition_num):
    topic_partition_expected = TopicPartition(topic=topic_name, partition=partition)
    topic_partition_actual, data, _ = topic.read(partition=partition, offset=0)

    # Read returns correct TopicPartition key
    assert topic_partition_actual == topic_partition_expected

    # Data is written into partition
    assert len(data) > 0

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 2

topic = KafkaTopic(topic_name, partition_num)

# write to topic with defined partition
for msg in msgs:
    topic.write(msg, partition=0)

lengths = [len(topic.read(partition=i, offset=0)[1]) for i in range(partition_num)]

assert [1000, 0] == lengths

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 3

topic = KafkaTopic(topic_name, partition_num)

# write to topic with defined key
for msg in msgs[:450]:
    topic.write(msg, key=b"some_key")

for msg in msgs[450:]:
    topic.write(msg, key=b"some_key443")

lengths = [len(topic.read(partition=i, offset=0)[1]) for i in range(partition_num)]

assert [0, 450, 550] == sorted(lengths)

## Group metadata

In [ ]:
# | export


def split_list(list_to_split: List[Any], split_size: int) -> List[List[Any]]:
    """
    Split a list into smaller lists of a specified size.

    Args:
        list_to_split: The list to split.
        split_size: The size of each split.

    Returns:
        A list of smaller lists.
    """
    return [
        list_to_split[start_index : start_index + split_size]
        for start_index in range(0, len(list_to_split), split_size)
    ]

In [ ]:
assert split_list([1, 2, 3, 4, 5], 1) == [[1], [2], [3], [4], [5]]
assert split_list([1, 2, 3, 4, 5], 2) == [[1, 2], [3, 4], [5]]
assert split_list([1, 2, 3, 4, 5], 3) == [[1, 2, 3], [4, 5]]
assert split_list([1, 2, 3, 4, 5], 5) == [[1, 2, 3, 4, 5]]

In [ ]:
# | export


class GroupMetadata:
    def __init__(self, num_partitions: int):
        """
        Initialize a GroupMetadata object.

        Args:
            num_partitions: The number of partitions in the group.
        """
        self.num_partitions = num_partitions
        self.partitions_offsets: Dict[int, int] = {}
        self.consumer_ids: List[uuid.UUID] = list()
        self.partition_assignments: Dict[uuid.UUID, List[int]] = {}

    def subscribe(self, consumer_id: uuid.UUID) -> None:
        """
        Subscribe a consumer to the group.

        Args:
            consumer_id: The ID of the consumer.
        """
        self.consumer_ids.append(consumer_id)
        self.rebalance()

    def unsubscribe(self, consumer_id: uuid.UUID) -> None:
        """
        Unsubscribe a consumer from the group.

        Args:
            consumer_id: The ID of the consumer.
        """
        self.consumer_ids.remove(consumer_id)
        self.rebalance()

    def rebalance(self) -> None:
        """
        Rebalance the group's partition assignments.
        """
        if len(self.consumer_ids) == 0:
            self.partition_assignments = {}
        else:
            partitions_per_actor = self.num_partitions // len(self.consumer_ids)
            if self.num_partitions % len(self.consumer_ids) != 0:
                partitions_per_actor += 1
            self.assign_partitions(partitions_per_actor)

    def assign_partitions(self, partitions_per_actor: int) -> None:
        partitions = [i for i in range(self.num_partitions)]

        partitions_split = split_list(partitions, partitions_per_actor)
        self.partition_assignments = {
            self.consumer_ids[i]: partition_split
            for i, partition_split in enumerate(partitions_split)
        }

    def get_partitions(
        self, consumer_id: uuid.UUID
    ) -> Tuple[List[int], Dict[int, Optional[int]]]:
        """
        Get the partition assignments and offsets for a consumer.

        Args:
            consumer_id: The ID of the consumer.

        Returns:
            A tuple containing the partition assignments and offsets.
        """
        partition_assignments = self.partition_assignments.get(consumer_id, [])
        partition_offsets_assignments = {
            partition: self.partitions_offsets.get(partition, None)
            for partition in partition_assignments
        }
        return partition_assignments, partition_offsets_assignments

    def set_offset(self, partition: int, offset: int) -> None:
        """
        Set the offset for a partition.

        Args:
            partition: The partition index.
            offset: The offset to set.
        """
        self.partitions_offsets[partition] = offset

In [ ]:
group_meta = GroupMetadata(num_partitions=3)

# subscribe first consumer
consumer_id_1 = uuid.uuid4()
group_meta.subscribe(consumer_id_1)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0, 1, 2]

# subscribe second consumer
consumer_id_2 = uuid.uuid4()
group_meta.subscribe(consumer_id_2)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0, 1]
assert group_meta.get_partitions(consumer_id_2)[0] == [2]

# subscribe third consumer
consumer_id_3 = uuid.uuid4()
group_meta.subscribe(consumer_id_3)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_3)[0] == [2]

# subscribe fourth consumer
# subscribe third consumer
consumer_id_4 = uuid.uuid4()
group_meta.subscribe(consumer_id_4)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_3)[0] == [2]
assert group_meta.get_partitions(consumer_id_4)[0] == []  # fourth consumer is starving

# Unsubscribe one consumer
group_meta.unsubscribe(consumer_id_3)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_4)[0] == [2], group_meta.get_partitions(
    consumer_id_4
)

# Unsubscribe all but one consumer
group_meta.unsubscribe(consumer_id_1)
group_meta.unsubscribe(consumer_id_4)
assert group_meta.get_partitions(consumer_id_2)[0] == [0, 1, 2]

## Kafka broker

In [ ]:
# | export


@classcontextmanager()
class InMemoryBroker:
    def __init__(
        self,
        num_partitions: int = 1,
    ):
        self.num_partitions = num_partitions
        self.topics: Dict[Tuple[str, str], KafkaTopic] = {}
        self.topic_groups: Dict[Tuple[str, str, str], GroupMetadata] = {}
        self.is_started: bool = False

    def connect(self) -> uuid.UUID:
        return uuid.uuid4()

    def dissconnect(self, consumer_id: uuid.UUID) -> None:
        """
        Disconnect a consumer from the broker.

        Args:
            consumer_id: The ID of the consumer.
        """
        pass

    def subscribe(
        self, bootstrap_server: str, topic: str, group: str, consumer_id: uuid.UUID
    ) -> None:
        raise NotImplementedError()

    def unsubscribe(
        self, bootstrap_server: str, topic: str, group: str, consumer_id: uuid.UUID
    ) -> None:
        raise NotImplementedError()

    def read(  # type: ignore
        self,
        *,
        bootstrap_server: str,
        topic: str,
        group: str,
        consumer_id: uuid.UUID,
        auto_offset_reset: str,
    ) -> Dict[TopicPartition, List[KafkaRecord]]:
        raise NotImplementedError()

    def write(  # type: ignore
        self,
        *,
        bootstrap_server: str,
        topic: str,
        value: bytes,
        key: Optional[bytes] = None,
        partition: Optional[int] = None,
    ) -> RecordMetadata:
        raise NotImplementedError()

    @contextmanager
    def lifecycle(self) -> Iterator["InMemoryBroker"]:
        """
        Context manager for the lifecycle of the in-memory broker.

        Yields:
            An instance of the in-memory broker.
        """
        raise NotImplementedError()

    async def _start(self) -> str:
        """
        Start the in-memory broker.

        Returns:
            The address of the broker.
        """
        logger.info("InMemoryBroker._start() called")
        self.__enter__()  # type: ignore
        return "localbroker:0"

    async def _stop(self) -> None:
        """
        Stop the in-memory broker.
        """
        logger.info("InMemoryBroker._stop() called")
        self.__exit__(None, None, None)  # type: ignore

In [ ]:
# | export


@patch
def subscribe(
    self: InMemoryBroker,
    bootstrap_server: str,
    topic: str,
    group: str,
    consumer_id: uuid.UUID,
) -> None:
    """
    Subscribe a consumer to a topic group.

    Args:
        bootstrap_server: The bootstrap server address.
        topic: The topic to subscribe to.
        group: The group to join.
        consumer_id: The ID of the consumer.
    """
    if (bootstrap_server, topic) not in self.topics:
        self.topics[(bootstrap_server, topic)] = KafkaTopic(
            topic=topic, num_partitions=self.num_partitions
        )

    group_meta = self.topic_groups.get(
        (bootstrap_server, topic, group), GroupMetadata(self.num_partitions)
    )
    group_meta.subscribe(consumer_id)
    self.topic_groups[(bootstrap_server, topic, group)] = group_meta


@patch
def unsubscribe(
    self: InMemoryBroker,
    bootstrap_server: str,
    topic: str,
    group: str,
    consumer_id: uuid.UUID,
) -> None:
    """
    Unsubscribe a consumer from a topic group.

    Args:
        bootstrap_server: The bootstrap server address.
        topic: The topic to unsubscribe from.
        group: The group to leave.
        consumer_id: The ID of the consumer.
    """
    self.topic_groups[(bootstrap_server, topic, group)].unsubscribe(consumer_id)

In [ ]:
topic = "topic1"
bootstrap_server = "localhost:9092"
consumer_group = "my_group"

broker = InMemoryBroker()

with pytest.raises(KeyError):
    broker.topic_groups[(bootstrap_server, topic, consumer_group)]

consumer_id = broker.connect()

broker.subscribe(bootstrap_server, topic, consumer_group, consumer_id)
broker.topic_groups[(bootstrap_server, topic, consumer_group)]

broker.unsubscribe(bootstrap_server, topic, consumer_group, consumer_id)

In [ ]:
# | export


@patch
def write(  # type: ignore
    self: InMemoryBroker,
    *,
    bootstrap_server: str,
    topic: str,
    value: bytes,
    key: Optional[bytes] = None,
    partition: Optional[int] = None,
) -> RecordMetadata:
    """
    Write a message to a topic.

    Args:
        bootstrap_server: The bootstrap server address.
        topic: The topic to write the message to.
        value: The value of the message.
        key: The key associated with the message.
        partition: The partition ID to write the message to.

    Returns:
        The metadata of the written message.
    """
    if (bootstrap_server, topic) not in self.topics:
        self.topics[(bootstrap_server, topic)] = KafkaTopic(
            topic=topic, num_partitions=self.num_partitions
        )

    return self.topics[(bootstrap_server, topic)].write(
        value, key=key, partition=partition
    )

In [ ]:
for key in [None, b"some_key"]:
    topic = "my_topic"
    bootstrap_server = "localhost:9092"
    value = b"msg"

    broker = InMemoryBroker(num_partitions=3)

    record_meta = broker.write(
        bootstrap_server=bootstrap_server, topic=topic, value=value, key=key
    )

    assert record_meta.topic == "my_topic"
    assert record_meta.offset == 0

    expected_msgs = [
        KafkaRecord(
            topic="my_topic",
            partition=record_meta.partition,
            key=key,
            value=b"msg",
            offset=0,
        )
    ]

    topic_partition, actual_msgs, new_offset = broker.topics[
        (bootstrap_server, topic)
    ].read(partition=record_meta.partition, offset=record_meta.offset)

    assert actual_msgs == expected_msgs
    assert topic_partition == record_meta.topic_partition
    assert new_offset == 1

In [ ]:
# | export


@patch
def read(  # type: ignore
    self: InMemoryBroker,
    *,
    bootstrap_server: str,
    topic: str,
    group: str,
    consumer_id: uuid.UUID,
    auto_offset_reset: str,
) -> Dict[TopicPartition, List[KafkaRecord]]:
    """
    Read messages from a topic group.

    Args:
        bootstrap_server: The bootstrap server address.
        topic: The topic to read messages from.
        group: The group to read messages for.
        consumer_id: The ID of the consumer.
        auto_offset_reset: The strategy to use when the consumer does not have a valid offset for the group.

    Returns:
        A dictionary containing the messages retrieved from each topic partition.
    """
    group_meta = self.topic_groups[(bootstrap_server, topic, group)]
    partitions, offsets = group_meta.get_partitions(consumer_id)

    if len(partitions) == 0:
        return {}

    partitions_data = {}

    for partition in partitions:
        offset = offsets[partition]

        if offset is None:
            offset = (
                self.topics[(bootstrap_server, topic)].latest_offset(partition)
                if auto_offset_reset == "latest"
                else 0
            )

        topic_partition, data, offset = self.topics[(bootstrap_server, topic)].read(
            partition, offset
        )

        partitions_data[topic_partition] = data
        group_meta.set_offset(partition, offset)

    return partitions_data

In [ ]:
# Check subscribing and reading from empty partitions for same group

topic = "topic1"
bootstrap_server = "localhost:9092"
consumer_group = "my_group"

broker = InMemoryBroker(num_partitions=3)

consumer_id_1 = broker.connect()
broker.subscribe(bootstrap_server, topic, consumer_group, consumer_id_1)

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    group=consumer_group,
    consumer_id=consumer_id_1,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic=topic, partition=0): [],
    TopicPartition(topic=topic, partition=1): [],
    TopicPartition(topic=topic, partition=2): [],
}

consumer_id_2 = broker.connect()
broker.subscribe(bootstrap_server, topic, consumer_group, consumer_id_2)

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    group=consumer_group,
    consumer_id=consumer_id_1,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic=topic, partition=0): [],
    TopicPartition(topic=topic, partition=1): [],
}

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    group=consumer_group,
    consumer_id=consumer_id_2,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic=topic, partition=2): [],
}

broker.unsubscribe(bootstrap_server, topic, consumer_group, consumer_id_1)
assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    group=consumer_group,
    consumer_id=consumer_id_2,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic=topic, partition=0): [],
    TopicPartition(topic=topic, partition=1): [],
    TopicPartition(topic=topic, partition=2): [],
}

In [ ]:
# check writing to partitions

topic = "topic1"
bootstrap_server = "localhost:9092"
consumer_group = "my_group"

broker = InMemoryBroker(num_partitions=1)

consumer_id_1 = broker.connect()
broker.subscribe(bootstrap_server, topic, consumer_group, consumer_id_1)

record_meta = broker.write(bootstrap_server=bootstrap_server, topic=topic, value=b"msg")

assert record_meta.topic == topic
assert record_meta.partition == 0
assert record_meta.topic_partition == TopicPartition(topic=topic, partition=0)
assert record_meta.offset == 0

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    consumer_id=consumer_id_1,
    group=consumer_group,
    auto_offset_reset="earliest",
) == {
    TopicPartition(topic=topic, partition=0): [
        KafkaRecord(topic=topic, partition=0, key=None, value=b"msg", offset=0)
    ]
}

broker.write(bootstrap_server=bootstrap_server, topic=topic, value=b"msg")

consumer_group_new = "another_group"

consumer_id_2 = broker.connect()
broker.subscribe(bootstrap_server, topic, consumer_group_new, consumer_id_2)

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    consumer_id=consumer_id_2,
    group=consumer_group_new,
    auto_offset_reset="latest",
) == {TopicPartition(topic=topic, partition=0): []}

assert broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    consumer_id=consumer_id_1,
    group=consumer_group,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic=topic, partition=0): [
        KafkaRecord(topic=topic, partition=0, key=None, value=b"msg", offset=1)
    ]
}

In [ ]:
topic = "my_topic"
bootstrap_server = "localhost:9092"
group = "my_group"

in_memory_broker = InMemoryBroker()

consumer_id = in_memory_broker.connect()

with pytest.raises(KeyError) as e:
    in_memory_broker.read(
        bootstrap_server=bootstrap_server,
        topic=topic,
        group=group,
        consumer_id=consumer_id,
        auto_offset_reset="latest",
    )

in_memory_broker.subscribe(
    bootstrap_server=bootstrap_server, topic=topic, group=group, consumer_id=consumer_id
)

msg = in_memory_broker.read(
    bootstrap_server=bootstrap_server,
    topic=topic,
    group=group,
    consumer_id=consumer_id,
    auto_offset_reset="earliest",
)
assert msg == {TopicPartition(topic=topic, partition=0): []}, msg

## Consumer patching

We need to patch AIOKafkaConsumer methods so that we can redirect the consumer to our local kafka broker.

Patched methods:

- [x] \_\_init\_\_
- [x] start
- [x] subscribe
- [x] stop
- [x] getmany

In [ ]:
# | export


# InMemoryConsumer
class InMemoryConsumer:
    def __init__(
        self,
        broker: InMemoryBroker,
    ) -> None:
        self.broker = broker
        self._id: Optional[uuid.UUID] = None
        self._auto_offset_reset: str = "latest"
        self._group_id: Optional[str] = None
        self._topics: List[str] = list()
        self._bootstrap_servers = ""

    @delegates(AIOKafkaConsumer)
    def __call__(self, **kwargs: Any) -> "InMemoryConsumer":
        defaults = _get_default_kwargs_from_sig(InMemoryConsumer.__call__, **kwargs)
        consume_copy = InMemoryConsumer(self.broker)
        consume_copy._auto_offset_reset = defaults["auto_offset_reset"]
        consume_copy._bootstrap_servers = (
            "".join(defaults["bootstrap_servers"])
            if isinstance(defaults["bootstrap_servers"], list)
            else defaults["bootstrap_servers"]
        )

        consume_copy._group_id = (
            defaults["group_id"]
            if defaults["group_id"] is not None
            else "".join(random.choices(string.ascii_letters, k=10))  # nosec
        )
        return consume_copy

    @delegates(AIOKafkaConsumer.start)
    async def start(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.stop)
    async def stop(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.subscribe)
    def subscribe(self, topics: List[str], **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaConsumer.getmany)
    async def getmany(  # type: ignore
        self, **kwargs: Any
    ) -> Dict[TopicPartition, List[ConsumerRecord]]:
        raise NotImplementedError()

In [ ]:
broker = InMemoryBroker()

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass, AIOKafkaConsumer]:
    consumer = cls()
    assert consumer._auto_offset_reset == "latest"

    consumer = cls(auto_offset_reset="earliest")
    assert consumer._auto_offset_reset == "earliest", consumer._auto_offset_reset

    consumer = cls(auto_offset_reset="whatever")
    assert consumer._auto_offset_reset == "whatever"

    await consumer.stop()

[ERROR] asyncio: Unclosed AIOKafkaConsumer
consumer: <aiokafka.consumer.consumer.AIOKafkaConsumer object>
[ERROR] asyncio: Unclosed AIOKafkaConsumer
consumer: <aiokafka.consumer.consumer.AIOKafkaConsumer object>


Patching start so that we don't try to start the real AIOKafkaConsumer instance

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.start)
async def start(self: InMemoryConsumer, **kwargs: Any) -> None:
    """
    Start consuming messages from the connected broker.

    Raises:
        RuntimeError: If start() has already been called without calling stop() first.
    """
    logger.info("AIOKafkaConsumer patched start() called()")
    if self._id is not None:
        raise RuntimeError(
            "Consumer start() already called! Run consumer stop() before running start() again"
        )
    self._id = self.broker.connect()

In [ ]:
broker = InMemoryBroker()

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass]:
    consumer = cls()
    await consumer.start()
    await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()


Patching subscribe so that we can connect to our Local, in-memory, Kafka broker

In [ ]:
# | export


@patch  # type: ignore
@delegates(AIOKafkaConsumer.subscribe)
def subscribe(self: InMemoryConsumer, topics: List[str], **kwargs: Any) -> None:
    """
    Subscribe to a list of topics for consuming messages.

    Args:
        topics: A list of topics to subscribe to.

    Raises:
        RuntimeError: If start() has not been called before calling subscribe().
    """
    logger.info("AIOKafkaConsumer patched subscribe() called")
    if self._id is None:
        raise RuntimeError("Consumer start() not called! Run consumer start() first")
    logger.info(f"AIOKafkaConsumer.subscribe(), subscribing to: {topics}")
    for topic in topics:
        self.broker.subscribe(
            bootstrap_server=self._bootstrap_servers,
            consumer_id=self._id,
            topic=topic,
            group=self._group_id,  # type: ignore
        )
        self._topics.append(topic)

In [ ]:
broker = InMemoryBroker()

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()
await consumer.start()
consumer.subscribe(["my_topic"])

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']


Patching stop so that be dont break anything by calling the real AIOKafkaConsumer stop()

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.stop)
async def stop(self: InMemoryConsumer, **kwargs: Any) -> None:
    """
    Stop consuming messages from the connected broker.

    Raises:
        RuntimeError: If start() has not been called before calling stop().
    """
    logger.info("AIOKafkaConsumer patched stop() called")
    if self._id is None:
        raise RuntimeError("Consumer start() not called! Run consumer start() first")
    for topic in self._topics:
        self.broker.unsubscribe(
            bootstrap_server=self._bootstrap_servers,
            topic=topic,
            group=self._group_id,  # type: ignore
            consumer_id=self._id,
        )

In [ ]:
broker = InMemoryBroker()

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()

await consumer.start()
consumer.subscribe(["my_topic"])
await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


Patching getmany so that the messages are pulled from our Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.getmany)
async def getmany(  # type: ignore
    self: InMemoryConsumer, **kwargs: Any
) -> Dict[TopicPartition, List[ConsumerRecord]]:
    """
    Retrieve messages from the subscribed topics.

    Returns:
        A dictionary containing the retrieved messages from each topic partition.

    Raises:
        RuntimeError: If start() has not been called before calling getmany().
    """
    await asyncio.sleep(0)
    for topic in self._topics:
        return self.broker.read(
            bootstrap_server=self._bootstrap_servers,
            topic=topic,
            consumer_id=self._id,  # type: ignore
            group=self._group_id,  # type: ignore
            auto_offset_reset=self._auto_offset_reset,
        )

In [ ]:
broker = InMemoryBroker()

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass(auto_offset_reset="latest")

await consumer.start()

consumer.subscribe(["my_topic"])
await consumer.getmany()

await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


## Producer patching

We need to patch AIOKafkaProducer methods so that we can redirect the producer to our local kafka broker

- [x] \_\_init\_\_
- [x] start
- [x] stop
- [x] send

In [ ]:
# | export


class InMemoryProducer:
    def __init__(self, broker: InMemoryBroker, **kwargs: Any) -> None:
        self.broker = broker
        self.id: Optional[uuid.UUID] = None
        self._bootstrap_servers = ""

    @delegates(AIOKafkaProducer)
    def __call__(self, **kwargs: Any) -> "InMemoryProducer":
        defaults = _get_default_kwargs_from_sig(InMemoryConsumer.__call__, **kwargs)
        producer_copy = InMemoryProducer(self.broker)
        producer_copy._bootstrap_servers = (
            "".join(defaults["bootstrap_servers"])
            if isinstance(defaults["bootstrap_servers"], list)
            else defaults["bootstrap_servers"]
        )
        return producer_copy

    @delegates(AIOKafkaProducer.start)
    async def start(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.stop)
    async def stop(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.send)
    async def send(  # type: ignore
        self,
        topic: str,
        msg: bytes,
        key: Optional[bytes] = None,
        **kwargs: Any,
    ):
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.partitions_for)
    async def partitions_for(self, topic: str) -> List[int]:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer._partition)
    def _partition(
        self, topic: str, arg1: Any, arg2: Any, arg3: Any, key: bytes, arg4: Any
    ) -> int:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.create_batch)
    def create_batch(self) -> "MockBatch":
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.send_batch)
    async def send_batch(self, batch: "MockBatch", topic: str, partition: Any) -> None:
        raise NotImplementedError()

In [ ]:
producer_cls = InMemoryProducer(None)

producer = producer_cls()
assert producer._bootstrap_servers == "localhost"

producer = producer_cls(bootstrap_servers="kafka.airt.ai")
assert producer._bootstrap_servers == "kafka.airt.ai"

Patching AIOKafkaProducer start so that we mock the startup procedure of AIOKafkaProducer

In [ ]:
# | export


@patch  # type: ignore
@delegates(AIOKafkaProducer.start)
async def start(self: InMemoryProducer, **kwargs: Any) -> None:
    """
    Start the in-memory producer.

    Raises:
        RuntimeError: If start() has already been called without calling stop() first.
    """
    logger.info("AIOKafkaProducer patched start() called()")
    if self.id is not None:
        raise RuntimeError(
            "Producer start() already called! Run producer stop() before running start() again"
        )
    self.id = self.broker.connect()

In [ ]:
broker = InMemoryBroker()

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()

[INFO] __main__: AIOKafkaProducer patched start() called()


Patching AIOKafkaProducerStop so that we don't uniintentionally try to stop a real instance of AIOKafkaProducer

In [ ]:
# | export


@patch  # type: ignore
@delegates(AIOKafkaProducer.stop)
async def stop(self: InMemoryProducer, **kwargs: Any) -> None:
    """
    Stop the in-memory producer.

    Raises:
        RuntimeError: If start() has not been called before calling stop().
    """
    logger.info("AIOKafkaProducer patched stop() called")
    if self.id is None:
        raise RuntimeError("Producer start() not called! Run producer start() first")

In [ ]:
broker = InMemoryBroker()

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
await producer.stop()

[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaProducer patched stop() called


Patching AIOKafkaProducer send so that we redirect sent messages to Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaProducer.send)
async def send(  # type: ignore
    self: InMemoryProducer,
    topic: str,
    msg: bytes,
    key: Optional[bytes] = None,
    partition: Optional[int] = None,
    **kwargs: Any,
):  # asyncio.Task[RecordMetadata]
    """
    Send a message to the specified topic.

    Args:
        topic: The topic to send the message to.
        msg: The message to send.
        key: The key associated with the message (optional).
        partition: The partition to send the message to (optional).
        **kwargs: Additional arguments to be passed to AIOKafkaProducer.send().

    Returns:
        A task that resolves to the RecordMetadata of the sent message.

    Raises:
        RuntimeError: If start() has not been called before calling send().
    """
    if self.id is None:
        raise RuntimeError("Producer start() not called! Run producer start() first")

    record = self.broker.write(
        bootstrap_server=self._bootstrap_servers,
        topic=topic,
        value=msg,
        key=key,
        partition=partition,
    )

    async def _f(record: ConsumerRecord = record) -> RecordMetadata:  # type: ignore
        return record

    return asyncio.create_task(_f())

In [ ]:
broker = InMemoryBroker()

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
msg_fut = await producer.send("my_topic", b"some_msg")
await msg_fut

[INFO] __main__: AIOKafkaProducer patched start() called()


RecordMetadata(topic='my_topic', partition=0, topic_partition=TopicPartition(topic='my_topic', partition=0), offset=0, timestamp=1680602752070, timestamp_type=0, log_start_offset=0)

In [ ]:
# | export


@patch
@delegates(AIOKafkaProducer.partitions_for)
async def partitions_for(self: InMemoryProducer, topic: str) -> List[int]:
    """
    Retrieve the list of partitions for the specified topic.

    Args:
        topic: The topic to get the partitions for.

    Returns:
        A list of partition IDs.
    """
    return [i for i in range(self.broker.num_partitions)]

In [ ]:
broker = InMemoryBroker(num_partitions=5)

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

assert len(await producer.partitions_for("some_topic")) == 5

In [ ]:
# | export


@patch
@delegates(AIOKafkaProducer._partition)
def _partition(
    self: InMemoryProducer,
    topic: str,
    arg1: Any,
    arg2: Any,
    arg3: Any,
    key: bytes,
    arg4: Any,
) -> int:
    """
    Determine the partition to which the message should be sent.

    Args:
        topic: The topic to send the message to.
        arg1, arg2, arg3, arg4: Additional arguments passed to the original AIOKafkaProducer._partition().

    Returns:
        The partition ID.
    """
    return int(hashlib.sha256(key).hexdigest(), 16) % self.broker.num_partitions

In [ ]:
broker = InMemoryBroker(num_partitions=5)

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

partition = producer._partition("my_topic", None, None, None, b"key", None)
assert partition >= 0 and partition < 5

In [ ]:
# | export


class MockBatch:
    def __init__(self) -> None:
        """
        Initialize an instance of MockBatch.
        """
        self._batch: List[Tuple] = list()

    def append(  # type: ignore
        self, key: Optional[bytes], value: bytes, timestamp: int
    ) -> RecordMetadata:
        """
        Append a message to the batch.

        Args:
            key: The key associated with the message (optional).
            value: The value of the message.
            timestamp: The timestamp of the message.

        Returns:
            The RecordMetadata of the appended message.
        """
        self._batch.append((key, value))
        return RecordMetadata(
            topic="",
            partition=0,
            topic_partition=None,
            offset=0,
            timestamp=timestamp,
            timestamp_type=0,
            log_start_offset=0,
        )


@patch
@delegates(AIOKafkaProducer.create_batch)
def create_batch(self: InMemoryProducer) -> "MockBatch":
    """
    Create a mock batch for the in-memory producer.

    Returns:
        A MockBatch instance.
    """
    return MockBatch()


@patch
@delegates(AIOKafkaProducer.send_batch)
async def send_batch(
    self: InMemoryProducer, batch: "MockBatch", topic: str, partition: Any
) -> None:
    """
    Send a batch of messages to the specified topic and partition.

    Args:
        batch: The MockBatch containing the messages to send.
        topic: The topic to send the batch of messages to.
        partition: The partition to send the batch of messages to.
    """
    for record in batch._batch:
        self.broker.write(
            bootstrap_server=self._bootstrap_servers,
            topic=topic,
            value=record[1],
            key=record[0],
            partition=partition,
        )

In [ ]:
bootstrap_server

broker = InMemoryBroker()

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()

batch = producer.create_batch()
batch.append(b"key", b"value", 1)

partition = producer._partition("my_topic", None, None, None, b"key", None)
await producer.send_batch(batch, topic, partition=partition)

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass(auto_offset_reset="earliest")

await consumer.start()

consumer.subscribe(["my_topic"])
msgs = await consumer.getmany()
assert len(msgs[TopicPartition(topic='my_topic', partition=0)]) == 1

[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']


In [ ]:
broker = InMemoryBroker()

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()

batch = producer.create_batch()
batch.append(b"key", b"value", 1)

partitions = await producer.partitions_for("my_topic")
partition = random.choice(tuple(partitions))

await producer.send_batch(batch, topic, partition=partition)

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass(auto_offset_reset="earliest")

await consumer.start()

consumer.subscribe(["my_topic"])
msgs = await consumer.getmany()
assert len(msgs[TopicPartition(topic='my_topic', partition=0)]) == 1

[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']


## Add patching to InMemoryBroker

In [ ]:
# | export


@patch
@contextmanager
def lifecycle(self: InMemoryBroker) -> Iterator[InMemoryBroker]:
    """
    Context manager for the lifecycle of the in-memory broker.

    Yields:
        An instance of the in-memory broker.
    """
    logger.info(
        "InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!"
    )
    try:
        logger.info("InMemoryBroker starting")

        old_consumer_app = fastkafka._application.app.AIOKafkaConsumer
        old_producer_app = fastkafka._application.app.AIOKafkaProducer
        old_consumer_loop = (
            fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer
        )

        fastkafka._application.app.AIOKafkaConsumer = InMemoryConsumer(self)
        fastkafka._application.app.AIOKafkaProducer = InMemoryProducer(self)
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = (
            InMemoryConsumer(self)
        )

        self.is_started = True
        yield self
    finally:
        logger.info("InMemoryBroker stopping")

        fastkafka._application.app.AIOKafkaConsumer = old_consumer_app
        fastkafka._application.app.AIOKafkaProducer = old_producer_app
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = (
            old_consumer_loop
        )

        self.is_started = False

In [ ]:
assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

with InMemoryBroker() as broker:
    assert isinstance(fastkafka._application.app.AIOKafkaConsumer, InMemoryConsumer)
    assert isinstance(fastkafka._application.app.AIOKafkaProducer, InMemoryProducer)
    assert fastkafka._application.app.AIOKafkaConsumer().broker == broker
    assert fastkafka._application.app.AIOKafkaProducer().broker == broker

assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: InMemoryBroker stopping


## Broker, consumer and producer integration tests

In [ ]:
@asynccontextmanager
async def create_consumer_and_producer(
    auto_offset_reset: str = "latest",
) -> AsyncIterator[Tuple[AIOKafkaConsumer, AIOKafkaProducer]]:
    consumer = fastkafka._application.app.AIOKafkaConsumer(
        auto_offset_reset=auto_offset_reset
    )
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer.start()
    await producer.start()

    yield (consumer, producer)

    await consumer.stop()
    await producer.stop()

In [ ]:
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [ ]:
assert checkEqual([1, 2], [3]) == False
assert checkEqual([1, 2, 3], [3, 2, 1]) == True

Sanity check, let's see if the messages are sent to broker and received by the consumer

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(320)]

with InMemoryBroker() as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic, msg) for msg in sent_msgs]
        consumer.subscribe([topic])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]
    assert checkEqual(
        received_msgs, sent_msgs
    ), f"{sent_msgs=}\n{received_msgs=}\n{data=}"

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check if only subscribed topic messages are received by the consumer

In [ ]:
topic1 = "test_topic1"
topic2 = "test_topic2"
sent_msgs_1 = [(f"msg{i}" + topic1).encode("UTF-8") for i in range(32)]
sent_msgs_2 = [(f"msg{i}" + topic2).encode("UTF-8") for i in range(32)]

with InMemoryBroker() as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic1, msg) for msg in sent_msgs_1]
        [await producer.send(topic2, msg) for msg in sent_msgs_2]

        consumer.subscribe([topic1])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_1, received_msgs)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic1']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check if msgs are received only after subscribing when auto_offset_reset is set to "latest"

In [ ]:
topic = "test_topic"
sent_msgs_before = [f"msg{i}".encode("UTF-8") for i in range(32)]
sent_msgs_after = [f"msg{i}".encode("UTF-8") for i in range(32, 64)]

with InMemoryBroker() as broker:
    async with create_consumer_and_producer() as (consumer, producer):
        [await producer.send(topic, msg) for msg in sent_msgs_before]

        consumer.subscribe([topic])
        received = await consumer.getmany()
        [await producer.send(topic, msg) for msg in sent_msgs_after]
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_after, received_msgs)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check two consumers different groups

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(32)]

with InMemoryBroker() as broker:
    consumer1 = fastkafka._application.app.AIOKafkaConsumer(
        auto_offset_reset="earliest"
    )
    consumer2 = fastkafka._application.app.AIOKafkaConsumer(
        auto_offset_reset="earliest"
    )
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer1.start()
    await consumer2.start()
    await producer.start()

    [await producer.send(topic, msg) for msg in sent_msgs]

    consumer1.subscribe([topic])
    received1 = await consumer1.getmany()

    consumer2.subscribe([topic])
    received2 = await consumer2.getmany()

    received_msgs1 = [msg.value for _, msgs in received1.items() for msg in msgs]
    received_msgs2 = [msg.value for _, msgs in received2.items() for msg in msgs]

    await consumer1.stop()
    await consumer2.stop()
    await producer.stop()

    assert checkEqual(sent_msgs, received_msgs1), received_msgs1
    assert checkEqual(sent_msgs, received_msgs2)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check two consumers same group

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(32)]

with InMemoryBroker(num_partitions=5) as broker:
    consumer1 = fastkafka._application.app.AIOKafkaConsumer(
        group_id="my_group", auto_offset_reset="earliest"
    )
    consumer2 = fastkafka._application.app.AIOKafkaConsumer(
        group_id="my_group", auto_offset_reset="earliest"
    )
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer1.start()
    await consumer2.start()
    await producer.start()

    [await producer.send(topic, msg) for msg in sent_msgs]

    consumer1.subscribe([topic])
    consumer2.subscribe([topic])

    received1 = await consumer1.getmany()
    received2 = await consumer2.getmany()

    received_msgs1 = [msg.value for _, msgs in received1.items() for msg in msgs]
    received_msgs2 = [msg.value for _, msgs in received2.items() for msg in msgs]

    await consumer1.stop()
    await consumer2.stop()
    await producer.stop()

    assert checkEqual(sent_msgs, received_msgs1 + received_msgs2)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check for different bootstrap servers

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(32)]

with InMemoryBroker() as broker:
    for server in ["localhost:9092", "kafka.airt.ai"]:
        consumer = fastkafka._application.app.AIOKafkaConsumer(
            bootstrap_servers=server, auto_offset_reset="earliest"
        )

        producer = fastkafka._application.app.AIOKafkaProducer(bootstrap_servers=server)

        await consumer.start()
        await producer.start()

        [await producer.send(topic, msg) for msg in sent_msgs]

        consumer.subscribe([topic])
        received = await consumer.getmany()

        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

        await consumer.stop()
        await producer.stop()

        assert checkEqual(sent_msgs, received_msgs)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping
